In [691]:
from collections import defaultdict
from enum import Enum
from typing import List, NamedTuple, Tuple, Dict

In [668]:
def get_lines(day: int) -> List[str]:
    with open(f"inputs/day{day}.txt", "r") as f:
        return f.read().splitlines()

# --- Day 1: Sonar Sweep ---


In [669]:
def count_increases(depths: List[int]) -> int:
    """Counts the number of times the depth increases"""

    return sum(d2 - d1 > 0 for d1, d2 in zip(depths, depths[1:]))

In [670]:
def sliding_sum(depths: List[int], window_size: int = 3) -> List[int]:
    """Computes a window_size-element sliding sum of the list of numbers"""

    views: List[List[int]] = [depths[i:] for i in range(window_size)]
    return [sum(window) for window in zip(*views)]

In [671]:
lines: List[str] = get_lines(1)
depths: List[int] = [int(line) for line in lines]

print(f"Answer #1: {count_increases(depths)}")
print(f"Answer #2: {count_increases(sliding_sum(depths))}")

Answer #1: 1532
Answer #2: 1571


# --- Day 2: Dive! ---


In [672]:
class Direction(Enum):
    FORWARD = 1
    DOWN = 2
    UP = 3

In [673]:
class Command(NamedTuple):
    direction: Direction
    magnitude: int

In [674]:
def parse_command(command: str) -> Command:
    """Parses a command assuming the format"""
    direction_mapping = {"forward": Direction.FORWARD, "down": Direction.DOWN, "up": Direction.UP}
    raw_direction, raw_magnitude = command.split(" ")
    
    return Command(direction_mapping[raw_direction], int(raw_magnitude))

In [675]:
def position_times_depth(commands: List[Command], use_aim=False) -> int:
    horizontal_position = 0
    depth = 0
    aim = 0

    for command in commands:
        match command, use_aim:
            case Command(Direction.FORWARD, x), True:
                horizontal_position += x
                depth += x * aim
            case Command(Direction.FORWARD, x), False:
                horizontal_position += x
            case Command(Direction.DOWN, x), True:
                aim += x
            case Command(Direction.DOWN, x), False:
                depth += x
            case Command(Direction.UP, x), True:
                aim -= x
            case Command(Direction.UP, x), False:
                depth -= x
    
    return horizontal_position * depth

In [676]:
lines: List[str] = get_lines(2)
commands: List[Command] = [parse_command(line) for line in lines]

print(f"Answer #1 {position_times_depth(commands)}")
print(f"Answer #2 {position_times_depth(commands, use_aim=True)}")

Answer #1 2120749
Answer #2 2138382217


# --- Day 3: Binary Diagnostic ---

In [677]:
def parse_reading(line: str) -> List[int]:
    return [int(x) for x in list(line)]

In [678]:
def count_values(readings: List[List[int]]) -> Dict[Tuple[int, int], int]:
    counter = defaultdict(int)

    for reading in readings:
        for i, value in enumerate(reading):
            counter[(i, value)] += 1

    return counter


In [679]:
def get_power_consumption(readings: List[List[int]]) -> int:
    epsilon_binary = []
    gamma_binary = []

    counter = count_values(readings)

    for i in range(len(readings[0])):
        most_common_value = max(0, 1, key=lambda value: counter[(i, value)])
        least_common_value = int(not most_common_value)
        
        epsilon_binary.append(str(most_common_value))
        gamma_binary.append(str(least_common_value))

    epsilon = int("".join(epsilon_binary), 2)
    gamma = int("".join(gamma_binary), 2)

    return epsilon * gamma


In [680]:
def get_life_support_rating(readings: List[List[int]]) -> int:    
    oxygen_candidates = readings
    co2_candidates = readings
    oxygen_rating = None
    co2_rating = None

    for i in range(len(readings[0])):
        oxygen_counter = count_values(oxygen_candidates)
        co2_counter = count_values(co2_candidates)
        o2_count_0 = oxygen_counter[(i, 0)]
        o2_count_1 = oxygen_counter[(i, 1)]
        co2_count_0 = co2_counter[(i, 0)]
        co2_count_1 = co2_counter[(i, 1)]

        if o2_count_0 == o2_count_1:
            oxygen_predicate = lambda x: x == 1
        else:
            oxygen_predicate = lambda x: x == max(0, 1, key=lambda v: oxygen_counter[(i, v)])

        if co2_count_0 == co2_count_1:
            co2_predicate = lambda x: x == 0
        else:
            co2_predicate = lambda x: x == min(0, 1, key=lambda v: co2_counter[(i, v)])

        oxygen_candidates = [reading for reading in oxygen_candidates if oxygen_predicate(reading[i])]
        co2_candidates = [reading for reading in co2_candidates if co2_predicate(reading[i])]

        if len(oxygen_candidates) == 1:
            oxygen_rating = int("".join(str(x) for x in oxygen_candidates[0]), 2)
        
        if len(co2_candidates) == 1:
            co2_rating = int("".join(str(x) for x in co2_candidates[0]), 2)

        if oxygen_rating is not None and co2_rating is not None:
            break

    return oxygen_rating * co2_rating


In [681]:
lines: List[str] = get_lines(3)
readings: List[List[int]] = [parse_reading(line) for line in lines]

print(f"Answer #1: {get_power_consumption(readings)}")
print(f"Answer #2: {get_life_support_rating(readings)}")

Answer #1: 3374136
Answer #2: 4432698


# --- Day 4: Giant Squid ---

In [682]:
Board = List[List[Tuple[int, bool]]]

In [683]:
def parse_nums(raw_nums: str) -> List[int]:
    return [int(num) for num in raw_nums.split(",")]

In [684]:
def parse_boards(raw_boards: List[str]) -> List[Board]:
    return [[[(int(x), False) for x in row.split(" ") if x] for row in board.split("\n") if row] for board in raw_boards]

In [685]:
def rows_solved(board: Board) -> bool:
    for row in board:
        if all(marked for marked in row):
            return True
    return False

In [686]:
def board_solved(board: Board) -> bool:
    marked_board = [[marked for _, marked in row] for row in board]
    transposed_board = list(map(list, zip(*marked_board)))
    return rows_solved(marked_board) or rows_solved(transposed_board)

In [687]:
def update_board(board: Board, drawn_num: int) -> Board:
    updated_board = []
    for row in board:
        updated_row = []
        for num, marked in row:
            updated_row.append((num, drawn_num == num or marked))
        updated_board.append(updated_row)
    
    return updated_board

In [688]:
def compute_bingo_score(board: Board, num: int) -> int:
    unmarked_sum = sum(x for row in board for x, marked in row if not marked)

    return unmarked_sum * num

In [692]:
def solve_bingo(drawn_nums: List[int], boards: List[Board], winner_first=True) -> int:
    updated_boards = boards
    solved_boards = []
    for num in drawn_nums:
        updated_boards = [update_board(board, num) for board in updated_boards if board not in solved_boards]
        for board in updated_boards:
            if board_solved(board):
                if winner_first:
                    return compute_bingo_score(board, num)
                solved_boards.append(board)
                if len(solved_boards) == len(boards):
                    return compute_bingo_score(board, num)

In [690]:
with open("inputs/day4.txt", "r") as f:
    raw_nums, *raw_boards = f.read().split("\n\n")
    
    nums = parse_nums(raw_nums)
    boards = parse_boards(raw_boards)

print(f"Answer #1: {solve_bingo(nums, boards)}")
print(f"Answer #2: {solve_bingo(nums, boards, winner_first=False)}")

Answer #1: 35670
Answer #2: 22704
